In [2]:
# Part 1: Setup and get a dataframe
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
# Initialize Pyspark
spark = SparkSession.builder.appName('Practice').getOrCreate()

spark

In [10]:
# Create a dataframe from a csv. and print data if wanted.
# Option 1
df_pyspark = spark \
                .read \
                .option('header', 'true') \
                .csv('test.csv', inferSchema=True)
# Option 2
df_pyspark = spark.read.csv('test.csv', header=True, inferSchema=True)

In [12]:
# Part2
#f_pyspark.select('name').show()
#df_pyspark.select(['name','age']).show()
#df_pyspark.describe().show()
#df_pyspark.dtypes

# Adding column in df
df_pyspark = df_pyspark.withColumn('experience', df_pyspark['age'] - 10)

# Drop it
#df_pyspark = df_pyspark.drop('experience')

# Rename
#df_pyspark.withColumnRenamed('experience', 'experience again').show()NNb 
df_pyspark = df_pyspark.drop('experience')
df_pyspark.show()

+-----+---+---+------+
| name|age|exp|salary|
+-----+---+---+------+
| sean| 28|  1|  null|
| jake| 11|  3| 20000|
|henry| 40| 20|100000|
+-----+---+---+------+



# Dealing with Null Values and Simple Imputer

In [17]:
# how="any" Drops rows with any na.
# thresh=2 at least 2 non null values should be present, then it drops any.
df_pyspark.na.drop("any", thresh=2)#.show()

# subset will drop any row with a NA  inside that column
df_pyspark.na.drop("any", subset=["salary"])#.show()


DataFrame[name: string, age: int, exp: int, salary: int]

In [25]:
# Filling values with the first parameter.
# you an subset with the second for any column
df_pyspark.na.fill(42, ["salary"]).show()

# More importantly, we might want to fill it with a mean using imputer:
from pyspark.ml.feature import Imputer

# NOTE: You can do multiple..
imputer = Imputer(
    inputCols=["salary", "age"],
    outputCols=[f"{x}_imputed" for x in ["salary", "age"]]
).setStrategy("mean")

imputer.fit(df_pyspark).transform(df_pyspark).show()

+-----+---+---+------+
| name|age|exp|salary|
+-----+---+---+------+
| sean| 28|  1|    42|
| jake| 11|  3| 20000|
|henry| 40| 20|100000|
+-----+---+---+------+

+-----+---+---+------+--------------+-----------+
| name|age|exp|salary|salary_imputed|age_imputed|
+-----+---+---+------+--------------+-----------+
| sean| 28|  1|  null|         60000|         28|
| jake| 11|  3| 20000|         20000|         11|
|henry| 40| 20|100000|        100000|         40|
+-----+---+---+------+--------------+-----------+

